In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate
from hmpai.pytorch.utilities import DEVICE, set_global_seed
from hmpai.normalization import *
from torchinfo import summary
from hmpai.utilities import print_results

In [2]:
set_global_seed(42)
data_path = Path("../data/sat1/split_stage_data.nc")
# data_path = Path("../data/sat1/split_stage_data_unprocessed_500hz.nc")

dataset = xr.load_dataset(data_path)
train_data, val_data, test_data = split_data_on_participants(dataset, 60, norm_dummy)

In [ ]:
train_kwargs = {"logs_path": Path("../logs/")}
model_kwargs = {
    "n_channels": len(dataset.channels),
    "n_samples": len(dataset.samples),
    "n_classes": len(dataset.labels),
}
results = k_fold_cross_validate(
    SAT1Base,
    model_kwargs,
    dataset,
    5,
    batch_size=128,
    normalization_fn=norm_dummy,
    train_kwargs={
        "logs_path": Path("../logs"),
        "additional_name": "CVtest",
    },
)
print_results(results)

In [10]:
model = SAT1GRU(
    len(train_data.channels), len(train_data.samples), len(train_data.labels)
)
train_and_test(model, train_data, test_data, val_data, logs_path=Path("../logs/"))

  0%|          | 0/758 [00:00<?, ? batch/s]

ERROR:tornado.general:SEND Error: Host unreachable


In [ ]:
summary(model, (16, 1, 147, 30))